Based on run.py and model.py

In [53]:
import argparse
import os
from datetime import date as datemethod
from datetime import datetime
import mesa_SIR.calculations_and_plots as c_p

import mesa
from mesa import Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import agent
from mesa_SIR import SIR

In [54]:
class custom_COVID_model(Model):
    
    def __init__(self, interactions, population, ptrans, reinfection_rate, I0, severe, progression_period, progression_sd, death_rate, recovery_days, recovery_sd):
        super().__init__(Model)
        
        self.susceptible = 0
        self.dead = 0
        self.recovered = 0
        self.infected = 0
        interactions = interactions
        self.population = population
        self.SIR_instance = SIR.Infection(self, ptrans = ptrans,
                                          reinfection_rate = reinfection_rate,
                                          I0= I0,
                                          severe = severe,
                                          progression_period = progression_period,
                                          progression_sd = progression_sd,
                                          death_rate = death_rate,
                                          recovery_days = recovery_days,
                                          recovery_sd = recovery_sd)


        G = SIR.build_network(interactions, self.population)
        self.grid = NetworkGrid(G)
        self.schedule = RandomActivation(self)
        self.dead_agents = []
        self.running = True
    
        for node in range(self.population):
            new_agent = agent.human(node, self) #what was self.next_id()
            self.grid.place_agent(new_agent, node)
            self.schedule.add(new_agent)

        #self.meme = 0
        self.datacollector = DataCollector(model_reporters={"infected": lambda m: c_p.compute(m,'infected'),
                                                            "recovered": lambda m: c_p.compute(m,'recovered'),
                                                            "susceptible": lambda m: c_p.compute(m,"susceptible"),
                                                            "dead": lambda m: c_p.compute(m, "dead"),
                                                            "R0": lambda m: c_p.compute(m, "R0"),
                                                            "severe_cases": lambda m: c_p.compute(m,"severe")})
        self.datacollector.collect(self)
    
    def step(self):
        self.schedule.step()
        
        self.datacollector.collect(self)
        '''
        for a in self.schedule.agents:
            if a.alive == False:
                self.schedule.remove(a)
                self.dead_agents.append(a.unique_id)
        '''

        if self.dead == self.schedule.get_agent_count():
            self.running = False
        else:
            self.running = True

In [55]:
# set params

# hardcode some params
population=4600 # population = Total population within all containers
steps=100 # steps = number of days in simulation
severe=0.05 # severe = Probability of developing severe, symptomatic disease
death_rate=0.001 # death_rate = Probability of dying after being infected after progression_period and before recovery_days
reinfection_rate=0.00 # reinfection_rate = Probability of becoming susceptible again after recovery
recovery_days=21 # recovery_days = Average number of days until recovery
recovery_sd=7 # recovery_sd = Standard deviation of recovery_days
I0=0.001 # I0 = Initial probability of being infected
progression_sd = 1 # progression_sd = Standard deviation of progression_period.

In [56]:
ptrans_tests = [0.01, 0.05, 0.1, 0.2, 0.25] # ptrans = Transmission probability
#progression_period_tests = [1, 2, 3] # progression_period = Average number of days until a patient seeks treatment.
progression_period_tests = [4, 5, 7] # progression_period = Average number of days until a patient seeks treatment.
interactions_tests = [0.25, 0.5, 1, 2] # interactions = Average number of interactions per person per day (decreases with social distancing)

In [57]:
for ptrans in ptrans_tests:
    for progression_period in progression_period_tests:
        for interactions in interactions_tests:
            param_path = str(ptrans) + '_' + str(progression_period) + '_' + str(interactions)
            
            # Instantiate model
            meme_model = custom_COVID_model(interactions, population, ptrans, reinfection_rate, I0, severe, progression_period, progression_sd, death_rate, recovery_days, recovery_sd)
            print('making model for: ' + param_path)
            for i in range(steps):
                meme_model.step()
            
            # Generate output    
            output_data = meme_model.datacollector.get_model_vars_dataframe()
            
            # define the name of the directory to be created
            filename = 'COVID_output_' + param_path + '.csv'
            path = '../out/' + param_path

            try:
                os.mkdir(path)
            except OSError:
                print ("Creation of the directory %s failed" % path)
                
            c_p.save_data(output_data, output_path = path, filename=filename)
            c_p.plot_SIR(output_data, param_path, path)
            c_p.plot_R0(output_data, param_path, path)
            c_p.plot_severe(output_data, param_path, path)

making model for: 0.01_4_0.25
making model for: 0.01_4_0.5
making model for: 0.01_4_1
making model for: 0.01_4_2
making model for: 0.01_5_0.25
making model for: 0.01_5_0.5
making model for: 0.01_5_1
making model for: 0.01_5_2
making model for: 0.01_7_0.25
making model for: 0.01_7_0.5
making model for: 0.01_7_1
making model for: 0.01_7_2
making model for: 0.05_4_0.25
making model for: 0.05_4_0.5
making model for: 0.05_4_1
making model for: 0.05_4_2
making model for: 0.05_5_0.25
making model for: 0.05_5_0.5
making model for: 0.05_5_1
making model for: 0.05_5_2
making model for: 0.05_7_0.25
making model for: 0.05_7_0.5
making model for: 0.05_7_1
making model for: 0.05_7_2
making model for: 0.1_4_0.25
making model for: 0.1_4_0.5
making model for: 0.1_4_1
making model for: 0.1_4_2
making model for: 0.1_5_0.25
making model for: 0.1_5_0.5
making model for: 0.1_5_1
making model for: 0.1_5_2
making model for: 0.1_7_0.25
making model for: 0.1_7_0.5
making model for: 0.1_7_1
making model for: 0.1